In [8]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  MoteInterArrivalTime is running.                  /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import numpy as np

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _SLACK_SYSTEMCHANNEL_NAME = config['analytics']['systemchannelname']
  _SLACK_SYSTEMCHANNEL_DBID = int(config['analytics']['systemchannelid'])

  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  MoteInterArrivalTime is running.                  /
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [9]:
def postMessageToSlack(text):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "packetstats"
    response = sc.api_call("chat.postMessage", channel=slackchannel, text=text, blocks=[])
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(response)
    else:
      print("Ok posting message to Slack channel")    

*Packet Inter-Arrival Time*

For all actively deployed LoRa Motes

In [10]:
#_LOG_LEVEL = _LOG_DEBUG
_INTERVAL_MINUTES = 60

_PREFERRED_IMAGE = "https://www.dropbox.com/s/ndxejw1xfd0x8z3/alert-icon.jpg?raw=1"

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE

# Reissue Alerts on OPEN tickets every 15minutes
_TICKETAGE_REISSUE_THRESHOLD_SECONDS = 15 * 60

# If mote is not seen for 30minutes, generate a ticket
_MAX_MOTE_AGE_SECONDS = 30 *60

#Ticket Type
MOTE_MISSING = 10003

#Mote types
_MAINS_MOTE_TYPE = 10000
_BATTERY_MOTE_TYPE = 10002

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

motequery = "SELECT * FROM kanji_node WHERE deploystate_id=10001 AND (nodetype_id={} OR nodetype_id={})".format(_MAINS_MOTE_TYPE, _BATTERY_MOTE_TYPE)
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of actively deployed LoRa Motes {}".format(len(df.index)))
slacktext = ""
squawking = 0
for ind in df.index:
    idmote = df['idnode'][ind]
    nameMote = df['name'][ind]
    specIatSecs = df['eventintervalsecs'][ind]
    logger(_LOG_DEBUG, idmote )
    moteeventquery = "SELECT timestamp, ack, timestamp - lag(timestamp, 1) OVER (ORDER BY timestamp) delta \
                  FROM kanji_eventlog WHERE node_id={} AND timestamp > NOW() - INTERVAL '{} MINUTES' \
                  ORDER BY timestamp desc".format(idmote, _INTERVAL_MINUTES)
    logger(_LOG_DEBUG, moteeventquery)
    df2 = pd.read_sql(moteeventquery, conn)
    df2['deltaseconds'] = df2['delta']/ np.timedelta64(1, 's')
    df2['diffseconds'] = df2['deltaseconds'] - specIatSecs
    eventcount = len(df2.index)
    
    logger(_LOG_DEBUG, df2.head(eventcount))
    logger(_LOG_DEBUG, "number of events in interval={}".format(eventcount))
    
    if len(df2.index)>1:
      squawking +=1
      ts2 = df2.set_index('timestamp')
      ts2.drop(ts2.index[len(df2.index)-1])
      #print(ts2.head(len(df2.index)-1))
      logger(_LOG_DEBUG, ts2.head(eventcount))
      ackMean = ts2.mean()['ack']
      iatMeanSecs = ts2.mean()['deltaseconds']  
      iatSdSecs   = ts2.std()['deltaseconds']
      iatDiffMeanSecs = ts2.mean()['diffseconds']
      iatDiffSdSecs   = ts2.std()['diffseconds']
      expectedEventCount = int(_INTERVAL_MINUTES * 60 / (specIatSecs + iatDiffMeanSecs))
      packetloss = (expectedEventCount - eventcount)/expectedEventCount 
      sdFromMean = (iatMeanSecs - specIatSecs)/iatSdSecs
      meanDiffAsRatioOfSpec = iatDiffMeanSecs/specIatSecs  
      #logger(_LOG_DEBUG, "mote {}, packet count={:3}, diffmean={:5.1f} ({:4.2f}spec) diffsd={:5.1f}".format(idmote, eventcount, iatDiffMeanSecs, meanDiffAsRatioOfSpec, iatDiffSdSecs))      
      #slacktext += "Mote {}, packets={:3}, loss={:3.3f} meanIat={:5.1f} diffmean={:5.1f}sec ({:4.2f}spec) diffsd={:5.1f}sec\n".format(nameMote, eventcount, packetloss, iatMeanSecs, iatDiffMeanSecs, meanDiffAsRatioOfSpec, iatDiffSdSecs) 
      slacktext += "{}, packets={:3}, loss={:3.3f} meanIAT={}secs meanAck={:3.3f}\n".format(nameMote, eventcount, packetloss, int(iatMeanSecs), ackMean) 
    else:
      slacktext += "{} NOT SQUAWKING!\n".format(nameMote)

slacktext = "{} of {} motes are squawking:\n".format(squawking, len(df.index)) + slacktext
logger(_LOG_INFO, slacktext)
postMessageToSlack(slacktext)    
logger(_LOG_INFO, "\nMoteInterArrivalTime Done!")  

5 of 6 motes are squawking:
agMote-20002, packets=187, loss=0.000 meanIAT=19secs meanAck=0.326
agMote-20009, packets=141, loss=0.066 meanIAT=23secs meanAck=0.255
agMote-20010, packets=149, loss=0.000 meanIAT=24secs meanAck=0.275
agSound, packets= 90, loss=0.250 meanIAT=29secs meanAck=0.233
agMote-20000 NOT SQUAWKING!
agMote-20003, packets=151, loss=0.000 meanIAT=23secs meanAck=0.298

Ok posting message to Slack channel

MoteInterArrivalTime Done!
